In [1]:
from pathlib import Path
import os
import sys
import subprocess
import warnings


def export_foldxpath(path_to_export):
    #export PATH=$PATH:/path/to/folder
    if str(path_to_export) not in os.environ["PATH"]:
        os.environ["PATH"] += ":"+str(path_to_export)
        print("foldx path exported")
    else:
        print("foldx bin folder already in PATH")

def rotabase_symlink(rotabase_path):
    
    #rotabase symlink
    sym_rotabase = Path("rotabase.txt")
    if not sym_rotabase.is_symlink():
        sym_rotabase.symlink_to(rotabase_path)
        print("Symlink to rotabase.txt create on working dir")
    else:
        print("rotabase.txt symlink already exist on working dir")
    
class FoldX:
    
    def __init__(self,exe="foldx",verbose=False,**kwargs):
        
        self.exe = exe
        self.options = kwargs
        self.verbose = verbose
                
    def run(self):

        """   ********************************************
       ***                                      ***
       ***             FoldX 4 (c)              ***
       ***                                      ***
       ***     code by the FoldX Consortium     ***
       ***                                      ***
       ***     Jesper Borg, Frederic Rousseau   ***
       ***    Joost Schymkowitz, Luis Serrano   ***
       ***    Peter Vanhee, Erik Verschueren    ***
       ***     Lies Baeten, Javier Delgado      ***
       ***       and Francois Stricher          ***
       *** and any other of the 9! permutations ***
       ***   based on an original concept by    ***
       ***   Raphael Guerois and Luis Serrano   ***
       ********************************************

    FoldX program options:


    Basic OPTIONS:
      -v [ --version ] arg (=Version beta 4)
                                            print version string
      -h [ --help ]                         produce help message
      -c [ --command ] arg                  Choose your FoldX Command:

                                            AlaScan
                                            AnalyseComplex
                                            BuildModel
                                            CrystalWaters
                                            Dihedrals
                                            DNAContact
                                            DNAScan
                                            LoopReconstruction
                                            MetalBinding
                                            Optimize
                                            PDBFile
                                            PepX
                                            PositionScan
                                            PrintNetworks
                                            Pssm
                                            QualityAssessment
                                            ReconstructSideChains
                                            RepairPDB
                                            Rmsd
                                            SequenceDetail
                                            SequenceOnly
                                            Stability

      -f [ --config ] arg                   config file location
      -d [ --debug ] arg                    Debug, produces more output

    Generic OPTIONS:
      --pdb arg (="")
      --pdb-list arg (="")                  File with a list of PDB files
      --pdb-dir arg (="./")                 PDB directory
      --output-dir arg (="./")              OutPut directory
      --output-file arg (="")               OutPut file
      --queue arg                           cluster queue: fast, normal, infinity,
                                            highmem, all.q
      --clean-mode arg (=0)                 FoldX clean mode: none, all, output or
                                            pdb
      --max-nr-retries arg (=1)             Maximum number of retries of a FoldX
                                            command if not finished successfully.
                                            Especially important to set at least to
                                            two when working on a cluster and file
                                            transfers often fail.
      --skip-build arg (=0)                 Skip the build step in the algorithm

    FoldX OPTIONS:

    input:
      --fixSideChains arg                   allows FoldX to complete missing
                                            sidechains at read-time, defaults to
                                            true
      --rotabaseLocation arg                set the location of the rotabase,
                                            defaults to rotabase.txt
      --noCterm arg                         set whether the last residue in a list
                                            of peptides (ex:ABC) shouldn't be
                                            considered as the C-terminal (i.e.,
                                            have an OXT), defaults to none
      --noNterm arg                         set whether the first residue in a list
                                            peptides (ex: ABC) shouldn't be
                                            considered as the N-Terminal (i.e.,
                                            have a third proton on the N), defaults
                                            to none

    output:
      --screen arg (=1)                     sets screen output, defaults to true
      --overwriteBatch arg (=1)             set to overwrite or not the specific
                                            name given as the first value in a
                                            command, defaults to true
      --noHeader arg (=0)                   remove standard FoldX Header from
                                            outputs, defaults to false

    PDB output:
      --out-pdb arg (=1)                    set to output PDBs when doing
                                            mutations, defaults to true
      --pdbHydrogens arg (=0)               output the hydrogens we place in the
                                            generated pdbs, defaults to false
      --pdbWaters arg (=0)                  output the predicted water bridges in
                                            the generated pdbs, defaults to false
      --pdbIons arg (=0)                    output the predicted metal ions in the
                                            generated pdbs, defaults to false
      --pdbDummys arg (=0)                  output the the dummy atoms we use (for
                                            N and C caps of helixes as well as the
                                            free orbitals) in the generated pdbs,
                                            defaults to false
      --pdbIsoforms arg (=0)                output the isoforms of the His in the
                                            generated pdbs, defaults to false

    physico chemical parameters:
      --temperature arg                     set the temperature (in K) of the
                                            calculations, defaults to 298 K
      --pH arg                              set the pH of the calculations,
                                            defaults to 7
      --ionStrength arg                     set the ionic strength of the
                                            calculations, defaults to 0.05

    force-field:
      --vdwDesign arg                       set VdWDesign of the experiment,
                                            defaults to 2 ( 0 very soft, 1 medium
                                            soft, 2 strong used for design )
      --clashCapDesign arg                  set maximun penalty per atom of the van
                                            der waals' clashes, defaults set to 5.0
      --backBoneAtoms arg                   consider only backbone atoms for all
                                            energy calculations, defaults to false
      --dipoles arg                         set to consider helices dipoles,
                                            defaults to true
      --complexClashes arg                  set the threshold (in kcal/mol) for
                                            counting clashing aminoacids at the
                                            interface, defaults to 1.

    entropy calculations:
      --fullMainEntropy arg                 set to maximally penalize the main
                                            chain of ligand and protein (usefull
                                            when comparing peptide data with
                                            poly-Alanine backbones), defaults to
                                            false

    water and ion evaluations:
      --water arg                           set how FoldX considers waters:
                                            -CRYSTAL (read the pdb waters) -PREDICT
                                            (predict water bridges from sratch)
                                            -IGNORE (don't consider waters)
                                            -COMPARE, defaults to -IGNORE

    complex options:
      --complexWithDNA arg                  set to consider only two groups in a
                                            protein-DNA complex, DNA + protein,
                                            defaults to false

    algorithm specific parameters:
      --moveNeighbours arg                  set to move neighbours when we mutate,
                                            defaults to true
      --numberOfRuns arg                    set the number of runs done in
                                            BuidModel, defaults to 1
      --fitAtoms arg                        set atoms involved in the RMSD command
                                            BB(backbone atoms), CA(Calpha),
                                            CA_CB(both Calpha and Cbeta),
                                            N_CA_O(N,Calpha and O), defaults to BB
      --rmsdPDB arg                         print out the rotated target of the
                                            RMSD command, defaults to true
      --repair_Interface arg                set to limit RepairPDB when applying to
                                            a complex: ALL(repair all residues
                                            including interface), ONLY(repair only
                                            the interface), NONE(no repair of the
                                            interface), defaults to ALL
      --burialLimit arg                     set a burial limit under which a
                                            residue is not repaired, defaults to 1.
                                            (inactive)
      --bFactorLimit arg                    set a relative bFactor limit above
                                            which a residue is not repaired,
                                            defaults to 0. (inactive)"""


        cmd = [self.exe]
        if self.options:
            for key,value in self.options.items():
                cmd.extend(["--"+key,value])

        p = subprocess.Popen(cmd, universal_newlines=True,shell=False,
                             stdout=subprocess.PIPE)
        
        while True:
            out = p.stdout.readline()   
            if not out and p.poll() is not None:
                break
            if self.verbose and out:
                print(out.splitlines()[0])

# Testing foldx class

In [2]:

ABPRED_DIR = Path.cwd().parent
CWD = Path.cwd()
FOLDX_DIR = ABPRED_DIR/"libs/foldxLinux64"
#FOLDX_DIR = ABPRED_DIR/"libs/foldxMacC11"
ROTABASE = FOLDX_DIR/"rotabase.txt"

#PDBS TO REAPIR PATH
PDBS_DIR = CWD / "pdbs_to_repair"
#Exporting path
export_foldxpath(FOLDX_DIR)



foldx path exported


In [3]:
pdbs_to_repair = PDBS_DIR.glob("*.pdb")
try:
    os.chdir(PDBS_DIR)
    #create symlink to rotabase.txt
    rotabase_symlink(ROTABASE)
    (PDBS_DIR.glob("*.pdb"))
    for pdb in pdbs_to_repair:
        options = {"command":"RepairPDB","repair_Interface":"ONLY","pdb":str(pdb.name)}
        
        FoldX(exe="foldx",verbose=True,**options).run()
finally:
    os.chdir(CWD)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/vfica/Documents/GitHub/AbPred/notebooks/pdbs_to_repair'

In [2]:
subprocess.Popen?